# Running multiple planting dates for best yields

In [44]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import pcse

from pcse.fileinput import CABOFileReader #Importing CABOFileReader (crop and soil files are saved in CABO format)
from pcse.util import WOFOST80SiteDataProvider
from pcse.fileinput import YAMLCropDataProvider # for crop
from pcse.base import ParameterProvider # combine parameters
from pcse.fileinput import YAMLAgroManagementReader # for agromanagement
from pcse.fileinput import ExcelWeatherDataProvider # for weather
from pcse.models import Wofost72_WLP_FD, Wofost72_PP # import PCSE model

from pcse.exceptions import WeatherDataProviderError
from pcse.db import NASAPowerWeatherDataProvider
#from progressbar import printProgressBar

from itertools import product
import yaml
import datetime
from datetime import date, timedelta

%matplotlib inline


In [45]:
cropfile = os.path.join(os.getcwd(), 'data', 'crop')
soilfile = os.path.join(os.getcwd(), 'data', 'soil', 'M03.AWC')
agromanagement_file = os.path.join(os.getcwd(), 'data',  'agro', 'soybean_calendar.agro')
#weatherfile = os.path.join(os.getcwd(), 'data', 'weather', 'Kk1.xls') 



In [46]:
sited = WOFOST80SiteDataProvider(WAV=10, CO2=360)
        #print(sited)
        # Reading crop parameters    
cropd = YAMLCropDataProvider(fpath=cropfile)  #cropd.print_crops_varieties()    
cropd.set_active_crop('soybean', 'Soybean_SC_Saga') #Set crop type and variety  
        # Reading soil parameters
soild = CABOFileReader(soilfile)

params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)

In [29]:
# Weather data for Netherlands
longitude= 34.7
latitude = 0.2
weatherd = NASAPowerWeatherDataProvider(latitude=latitude, longitude=longitude)
#print(weatherd)

In [31]:

crop_name = 'soybean'
variety_name = 'Soybean_SC_Saga'
campaign_start_date = datetime.date(1990, 1, 1)
crop_start_date = campaign_start_date
max_duration = 106
crop_start_date = campaign_start_date
end_date = date(2012, 7, 31)
delta = end_date - campaign_start_date  # as timedelta

dates =[]
predictionsWLP = []
predictionsPP = []


for i in range(delta.days + 1):  
        crop_start_date = campaign_start_date + timedelta(days=i)
        crop_end_type = crop_start_date + timedelta(days=max_duration)
        # Define the agromanagement for the value chain      
        agro_yaml = """
        - {camsd}:
            CropCalendar:
                crop_name: '{cname}'
                variety_name: '{vname}'
                crop_start_date: {csd}
                crop_start_type: sowing
                crop_end_date: {ced}
                crop_end_type: harvest
                max_duration: {maxdur}
            TimedEvents: null
            StateEvents: null
        """
        agro = yaml.safe_load(agro_yaml.format(cname=crop_name, vname=variety_name, camsd=campaign_start_date, csd=crop_start_date, 
                                               ced=crop_end_type, maxdur=max_duration))
        #print(agro)
        wofost = Wofost72_PP(params, weatherd, agro)
        
        # Initialize PCSE model
        wofsimWLP = Wofost72_WLP_FD(params, weatherd, agro) # water limited production
        wofsimPP = Wofost72_PP(params, weatherd, agro) # potential production
        
        #Run Water Limited production
        wofsimWLP.run_till_terminate()
        df_resultsWLP = pd.DataFrame(wofsimWLP.get_output())
        df_resultsWLP = df_resultsWLP.set_index("day")
        
        #Run potential production
        wofsimPP.run_till_terminate()
        df_resultsPP = pd.DataFrame(wofsimPP.get_output())
        df_resultsPP = df_resultsPP.set_index("day")
        
        #Results of the simulation
        summary_outputWLP = wofsimWLP.get_summary_output()
        msgWLP = "WLP Reached maturity with total biomass of {TAGP:.2f} kg/ha and gross yield of {TWSO:.2f} kg/ha with a maximum LAI of {LAIMAX:.2f}."
 
        for crop_cycleWLP in summary_outputWLP:
            print(msgWLP.format(**crop_cycleWLP))
        predictionsWLP.append(crop_cycleWLP['TWSO'])
    
        summary_outputPP = wofsimPP.get_summary_output()
        msgPP = "PP Reached maturity with total biomass of {TAGP:.2f} kg/ha and gross yield of {TWSO:.2f} kg/ha with a maximum LAI of {LAIMAX:.2f}."

        for crop_cyclePP in summary_outputPP:
            print(msgPP.format(**crop_cyclePP))
        predictionsPP.append(crop_cyclePP['TWSO'])
        
        dates.append('{:%Y-%m-%d}'.format(crop_start_date))
        

#Export results
simulationResults= pd.DataFrame([dates, predictionsWLP, predictionsPP])
        
simulationResults.to_csv(r'simulationResults11092021.csv')     
    
    


WLP Reached maturity with total biomass of 9911.58 kg/ha and gross yield of 886.85 kg/ha with a maximum LAI of 10.07.
PP Reached maturity with total biomass of 17131.22 kg/ha and gross yield of 2003.59 kg/ha with a maximum LAI of 12.55.
WLP Reached maturity with total biomass of 10086.60 kg/ha and gross yield of 954.76 kg/ha with a maximum LAI of 10.11.
PP Reached maturity with total biomass of 17135.06 kg/ha and gross yield of 1994.77 kg/ha with a maximum LAI of 12.55.
WLP Reached maturity with total biomass of 10248.19 kg/ha and gross yield of 1013.79 kg/ha with a maximum LAI of 10.15.
PP Reached maturity with total biomass of 17120.03 kg/ha and gross yield of 1981.62 kg/ha with a maximum LAI of 12.54.
WLP Reached maturity with total biomass of 10412.91 kg/ha and gross yield of 1069.73 kg/ha with a maximum LAI of 10.18.
PP Reached maturity with total biomass of 17134.01 kg/ha and gross yield of 1970.66 kg/ha with a maximum LAI of 12.55.
WLP Reached maturity with total biomass of 1060